In [17]:
import numpy as np
import pandas as pd
import math
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

def printVar (varname, var):

    print(varname)
    print(var)

def saveArray (name, array):

    pd.DataFrame(array).to_csv("savedVariables/" + name + ".csv")



In [18]:
# Solve the 2D Navier-Cauchy equation using a segregated finite volume method

# Define mesh geometry and boundary conditions

Lx = .01 #width of domain
Ly = .01 #height of domain

fac = 1

u_left = fac*5e-6      #u boundary condition at the left boundary
v_left = fac*3e-6      #v boundary condition at the left boundary

u_right = fac*5e-6     #u boundary condition at the right boundary
v_right = fac*1e-6     #v boundary condition at the right boundary

u_top = fac*1e-6       #u boundary condition at the top boundary
v_top = fac*1e-6       #v boundary condition at the top boundary

u_bottom = fac*7e-6    #u boundary condition at the bottom boundary 
v_bottom = fac*1e-6    #v boundary condition at the bottom boundary

nx = 10            #number of control volumes in the x-direction
ny = 10            #number of control volumes in the y-direction


In [19]:
# Solve the 2D Navier-Cauchy equation using a segregated finite volume method

# Define mesh geometry and boundary conditions

# Lx = .01 #width of domain
# Ly = .01 #height of domain

# u_left = 0e-6      #u boundary condition at the left boundary
# v_left = 0e-6      #v boundary condition at the left boundary

# u_right = 0e-6     #u boundary condition at the right boundary
# v_right = 0e-6     #v boundary condition at the right boundary

# u_top = 0e-6       #u boundary condition at the top boundary
# v_top = 0e-6       #v boundary condition at the top boundary

# u_bottom = 0e-6    #u boundary condition at the bottom boundary 
# v_bottom = 0e-6    #v boundary condition at the bottom boundary

# nx = 10            #number of control volumes in the x-direction
# ny = 10            #number of control volumes in the y-direction


In [20]:
# Define material properties

rho = 8050         #density of steel in kg/m^3
mu = 78*10e9       #shear modulus in Pa
K = 1.6*10e11      #bulk modulus in Pa

Lambda = K - (2/3)*mu

print(Lambda)

1080000000000.0


In [21]:


## Define required variables

tf = 0.01              #total time in seconds
dx = Lx/nx          #length of each control volume in the x-direction
dy = Ly/ny          #length of each control volume in the y-direction
dt = tf/10         #size of time steps
Sfx = dy            #area vector x component (Area of East and West Faces)
Sfy = dx            #area vector y component (Area of North and South Faces)


In [22]:
## Define  position and time vectors

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
x = np.zeros((1,nx+2))   #position vector defining the center points of the control volumes in the x-direction.
x[0,nx+1] = Lx
x[0,1:nx+1] = np.arange(dx/2,Lx,dx)

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
y = np.zeros((1,ny+2))   #position vector defining the center points of the control volumes in the y-direction.
y[0,ny+1] = Ly
y[0,1:ny+1] = np.arange(dy/2,Ly,dy)

print(x)
print(y)


[[0.     0.0005 0.0015 0.0025 0.0035 0.0045 0.0055 0.0065 0.0075 0.0085
  0.0095 0.01  ]]
[[0.     0.0005 0.0015 0.0025 0.0035 0.0045 0.0055 0.0065 0.0075 0.0085
  0.0095 0.01  ]]


In [23]:
## Define required displacement matrices and b vector

u_new = np.zeros([nx*ny,2])         #unknown displacements at t + 1    (Column 1 = x, Column2 = y)
u_old = np.zeros([nx*ny,2])         #displacement at time t
u_old_old = np.zeros([nx*ny,2])     #displacement at time t - 1

#rhs vector
b_x = np.zeros([nx*ny,1])
b_y = np.zeros([nx*ny,1])

In [24]:
## Define coefficient matrix A with no boundary conditions

internalFaceNo = ny*(nx - 1) + nx*(ny - 1)    # Number of internal faces
nonZeroNo = nx*ny + 2*internalFaceNo
A_no_boundary = np.zeros([ny*nx, ny*nx])
A_no_boundary = csr_matrix(A_no_boundary)

# To display A_no_boundary:

print(A_no_boundary.todense().shape)

(100, 100)


## X equation:

In [25]:
# Only for x-equation:

a_P_x = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((2*mu)*Sfy)/dy

a_N_x = ((mu)*Sfy)/dy
a_S_x = ((mu)*Sfy)/dy
a_E_x = ((2*mu + Lambda)*Sfx)/dx
a_W_x = ((2*mu + Lambda)*Sfx)/dx


In [26]:
A_no_boundary_x = A_no_boundary

## Define internal coefficients of matrix A for values with no B.C.s for x-equation

#j is column number in mesh (y-coord)
#k is point number in mesh

for j in np.arange(1, ny-1):     #main diagonal, starting from y=1 because all values at y=0 have B.C.s

    for k in np.arange((j*nx)+1, (j*nx)+nx-1):  
        
        A_no_boundary_x[k,k] = a_P_x
        A_no_boundary_x[k,k+nx] = -a_N_x
        A_no_boundary_x[k,k-nx] = -a_S_x
        A_no_boundary_x[k,k+1] = -a_E_x
        A_no_boundary_x[k,k-1] = -a_W_x

        # print("\tN: " + str(k+nx), "\nW: " + str(k-1), "\tP: " + str(k), "\tE: " + str(k+1), "\n\tS: " + str(k-nx) + "\n")

/home/scottlevie/miniconda3/lib/python3.9/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



## y equation:

In [27]:
# Only for y-equation:

a_P_y = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfy)/dy + ((2*mu)*Sfx)/dx 

a_N_y = ((2*mu + Lambda)*Sfy)/dy
a_S_y = ((2*mu + Lambda)*Sfy)/dy
a_E_y = ((mu)*Sfx)/dx
a_W_y = ((mu)*Sfx)/dx


In [28]:
A_no_boundary_y = A_no_boundary

## Define internal coefficients of matrix A for values with no B.C.s for x-equation

#j is column number in mesh (y-coord)
#k is point number in mesh

for j in np.arange(1, ny-1):     #main diagonal, starting from y=1 because all values at y=0 have B.C.s

    for k in np.arange((j*nx)+1, (j*nx)+nx-1):  
        
        A_no_boundary_y[k,k] = a_P_y

        A_no_boundary_y[k,k+nx] = -a_N_y
        A_no_boundary_y[k,k-nx] = -a_S_y
        A_no_boundary_y[k,k+1] = -a_E_y
        A_no_boundary_y[k,k-1] = -a_W_y

        # print("\tN: " + str(k+nx), "\nW: " + str(k-1), "\tP: " + str(k), "\tE: " + str(k+1), "\n\tS: " + str(k-nx) + "\n")

In [29]:
## Time loop

!rm -r savedVariables
!mkdir savedVariables

for time in [1]:

    print("\nTime = " + str(time))
    
    # Set number of maximum iterations for convergence
    
    icorr = 1
    maxcorr = 100

    # Make directory

    !mkdir savedVariables/{time}
    
    # while True:
    for iteration in [1]:
            
        u_previous = u_new #store latest guess values
        
        #Counters will be used later for the center left and right boundary conditions.
        x_counter_left = 1
        x_counter_right = 2
        y_counter_left = 1
        y_counter_right = 2
        
        # Prepare matrix A and b vector for the x-equation
        
        A_x = A_no_boundary_x #make copy of matrix A with no boundary conditions
        
        for k in np.arange(0,nx*ny):   # k is the cell number

            #Bottom left corner coefficients
                
            if k == 0:

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*3*mu + (Sfx/dx)*(6*mu + 3*Lambda)

                #an
                A_x[k,k+nx] = -(Sfy/dy)*mu

                #ae
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                #b_x = A + B(ub) + C(ul) + D(ve + vne - 2vl) + E(vn + vne - 2vb)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            + (Sfy/dy)*(2*mu)*u_bottom
                            + (Sfx/dx)*(4*mu +2*Lambda)*u_left
                            + (Sfy/(3*dx))*mu*( 
                                                u_previous[k+1,1] 
                                                +  u_previous[k+1+nx, 1]
                                                + - 2*v_left
                                                )
                            + (Sfx/(3*dy))*Lambda*( 
                                                    + u_previous[k+nx , 1]
                                                    + u_previous[k+1+nx , 1]
                                                    - 2*v_bottom
                                                    )
                        )   

            #Bottom right corner coefficients
                
            elif  k == nx-1:

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*3*mu + (Sfx/dx)*(6*mu + 3*Lambda)

                #an
                A_x[k,k+nx] = -(Sfy/dy)*mu

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                #b_x = A + B(u_b) + C(u_r) + D(-v_w -v_nw + 2v_r) + E(-v_n -v_nw + 2v_b)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            + (Sfy/dy)*(2*mu)*u_bottom
                            + (Sfx/dx)*(4*mu +2*Lambda)*u_right
                            + (Sfy/(3*dx))*mu*( 
                                                - u_previous[k-1,1]      #v_w
                                                - u_previous[k-1+nx, 1]  #v_nw
                                                + 2*v_right              #v_r
                                                )
                            + (Sfx/(3*dy))*Lambda*( 
                                                    - u_previous[k+nx , 1]      #v_n
                                                    - u_previous[k-1+nx , 1]    #v_nw
                                                    + 2*v_bottom                #v_b
                                                    )
                        )

            # Top Left Corner Coeficients
                
            elif  k == (nx*ny)-(nx):
                
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*3*mu + (Sfx/dx)*(6*mu + 3*Lambda)  ##ap               
                
                #as
                A_x[k,k-nx] = -(Sfy/dy)*mu

                #ae                
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                #b_x = A + B(ut) + C(ul) + D(-ve -vse + 2vl) + E(-vs - vse + 2vt) 
                
                b_x[k] =( 
                            (rho/(dt**2))*(2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            + (Sfy/dy)*(2*mu)*u_top 
                            + (Sfx/dx)*(4*mu +2*Lambda)*u_left 
                            + (Sfy/(3*dx))*mu*( 
                                                    - u_previous[k+1,1] 
                                                    - u_previous[k+1-nx, 1] 
                                                    + 2*v_left
                                                ) 
                            + (Sfx/(3*dy))*Lambda*(
                                                    - u_previous[k-nx , 1]
                                                    - u_previous[k+1-nx , 1]
                                                    + 2*v_top
                                                    )    
                        )
                
            # Top right corner coefficients
                
            elif  k == (nx*ny)-1:

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*3*mu + (Sfx/dx)*(6*mu + 3*Lambda)

                #as
                A_x[k,k-nx] = -(Sfy/dy)*mu

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                #b_x = A + B(u_t) + C(u_r) + D(+ v_w + v_sw - 2v_r) + E(+ v_s + v_sw - 2v_t)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            + (Sfy/dy)*(2*mu)*u_bottom
                            + (Sfx/dx)*(4*mu +2*Lambda)*u_right
                            + (Sfy/(3*dx))*mu*( 
                                                + u_previous[k-1,1]      # + v_w
                                                + u_previous[k-1-nx, 1]  # + v_sw
                                                - 2*v_right              # - 2v_r
                                                )
                            + (Sfx/(3*dy))*Lambda*( 
                                                    + u_previous[k-nx , 1]      # + v_s
                                                    + u_previous[k-1-nx , 1]    # + v_sw
                                                    - 2*v_bottom                # - 2v_t
                                                    )
                        )
                
            # Center Bottom Boundaries

            elif  (k > 0) & (k < nx):

                # print("x - Bottom Center Coefficents for Cell: " + str(k))

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*3*mu + (Sfx/dx)*(4*mu + 2*Lambda)

                #an
                A_x[k,k+nx] = -(Sfy/dy)*mu
                
                #ae
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                #b_x = A + B(u_b) + C(v_ne - v_nw)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            + (Sfy/dy)*(2*mu)*u_bottom # u-b
                            + (
                                (Sfy/(4*dx))*mu 
                                +
                                (Sfx/(3*dy))*Lambda
                            )*(
                                + u_previous[k+1+nx, 1] #v_ne
                                - u_previous[k-1+nx, 1] #v_nw
                            )
                        )

            # Center Top Boundaries

            elif  ( k > (nx*ny)-(nx) ) & ( k < (nx*ny)-1 ):

                # print("x - Top Center Coefficents for Cell: " + str(k))

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*3*mu + (Sfx/dx)*(4*mu + 2*Lambda)

                #as
                A_x[k,k-nx] = -(Sfy/dy)*mu
                
                #ae
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                #b_x = A + B(u_t) + C(-v_se + v_sw)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            + (Sfy/dy)*(2*mu)*u_top # u_t
                            + (
                                (Sfy/(4*dx))*mu 
                                +
                                (Sfx/(3*dy))*Lambda
                            )*(
                                - u_previous[k+1-nx, 1] # - v_se
                                + u_previous[k-1-nx, 1] # + v_sw
                            )
                        )
            
            # Center Left Boundaries

            elif  ( k > 0) & ( k%nx == 0 ) & ( k < (nx*ny)-(nx) ):

                # print("x - Left Center Coefficents for Cell: " + str(k))

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*2*mu + (Sfx/dx)*(6*mu + 3*Lambda)

                #an
                A_x[k,k+nx] = -(Sfy/dy)*mu
                
                #as
                A_x[k,k-nx] = -(Sfy/dy)*mu

                #ae
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                #b_x = A + B(u_l) + C( + v_ne - v_se)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            + (Sfx/dx)*(4*mu + 2*Lambda)*u_left # u_l
                            + (
                                (Sfy/(3*dx))*mu 
                                +
                                (Sfx/(4*dy))*Lambda
                            )*(
                                + u_previous[k+1+nx, 1] # + v_ne
                                - u_previous[k+1-nx, 1] # - v_se
                            )
                        )

            # Center Right Boundaries

            elif  ( k > nx - 1) & ( k%(nx) == nx - 1 ) & ( k < nx*ny -1 ):

                # print("x - Right Center Coefficents for Cell: " + str(k))

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*2*mu + (Sfx/dx)*(6*mu + 3*Lambda)

                #an
                A_x[k,k+nx] = -(Sfy/dy)*mu
                
                #as
                A_x[k,k-nx] = -(Sfy/dy)*mu

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                #b_x = A + B(u_r) + C( - v_nw + v_sw)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            + (Sfx/dx)*(4*mu + 2*Lambda)*u_right # u_l
                            + (
                                (Sfy/(3*dx))*mu 
                                +
                                (Sfx/(4*dy))*Lambda
                            )*(
                                - u_previous[k-1+nx, 1] # - v_nw
                                + u_previous[k-1-nx, 1] # + v_sw
                            )
                        )
            
            else:

                b_x[k] = (
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy)
                            + (Sfy/(4*dx))*mu*(
                                                + u_previous[k+1+nx,1]    # + v_ne
                                                - u_previous[k-1+nx,1]    # - v_nw
                                                - u_previous[k+1-nx,1]    # - v_se
                                                + u_previous[k-1-nx,1]    # + v_sw
                                            )
                            +(Sfx/(4*dy))*Lambda*(
                                                    + u_previous[k+1+nx,1]    # + v_ne
                                                    - u_previous[k-1+nx,1]    # - v_nw
                                                    - u_previous[k+1-nx,1]    # - v_se
                                                    + u_previous[k-1-nx,1]    # + v_sw 
                                                )
                            )
        saveArray(str(time) + "/A_x_" + str(icorr) , A_x.todense())   
        saveArray(str(time) + "/b_x_" + str(icorr), b_x)     
        
        
        ## Solve the x-equation
        
        Ux = spsolve(A_x, b_x)
        Ux = np.array(Ux)

        saveArray(str(time) + "/Ux_" + str(icorr), Ux)
        
        ## Prepare matrix A and b vector for the y-equation
        
        A_y = A_no_boundary_y #make copy of A with no boundary conditions
        
        for k in np.arange(0,nx*ny):   # j is the cell number
            
            #Bottom left corner coefficients
            
            if k == 0:
                
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(6*mu + 3*Lambda) + (Sfx/dx)*(3*mu)  ##ap       (6*mu + 3*Lambda)        
                
                #an
                A_y[k,k+nx] = -(Sfy/dy)*(2*mu + Lambda)

                #ae                
                A_y[k,k+1] = -(Sfx/dx)*(mu)

                # b_y = A + B(vb) + C(vl) + D(ue + une - 2ul) + E(un + une - 2ub) 
                
                b_y[k] = (
                                (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy)
                                + (Sfy/dy)*(4*mu +2*Lambda)*v_bottom
                                + (Sfx/dx)*(2*Lambda)*v_left
                                + (Sfy/(3*dx))*Lambda*( 
                                                        + u_previous[k+1,0] 
                                                        + u_previous[k+1+nx, 0]
                                                        - 2*u_left
                                                        ) 
                                + (Sfx/(3*dy))*mu*(
                                                    + u_previous[k+nx,0]
                                                    + u_previous[k+1+nx, 0]
                                                    - 2*u_bottom
                                                    )
                            )
                
            #Bottom right corner coefficients

            elif k == nx-1:
                
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(6*mu + 3*Lambda) + (Sfx/dx)*(3*mu)  ##ap       (6*mu + 3*Lambda)        
                
                #an
                A_y[k,k+nx] = -(Sfy/dy)*(2*mu + Lambda)

                #aw                
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                #b_y = A + B(vb) + C(vr) + D(- u_w - u_nw + 2u_r) + E(- u_n - u_nw + 2u_b) 
                
                b_y[k] = (
                                (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy)
                                + (Sfy/dy)*(4*mu +2*Lambda)*v_bottom
                                + (Sfx/dx)*(2*Lambda)*v_right
                                + (Sfy/(3*dx))*Lambda*( 
                                                        - u_previous[k-1,0]        # - u_w
                                                        - u_previous[k-1+nx, 0]    # - u_nw
                                                        + 2*u_right                # + 2u_r
                                                        ) 
                                + (Sfx/(3*dy))*mu*(
                                                    - u_previous[k+nx,0]        # - u_n 
                                                    - u_previous[k-1+nx, 0]     # - u_nw 
                                                    + 2*u_bottom                # + 2u_b
                                                    ) 
                            )
                     
            #Top left corner coefficients
                
            elif k == (nx*ny)-(nx):
                
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(6*mu + 3*Lambda) + (Sfx/dx)*(3*mu)  ##ap       (6*mu + 3*Lambda)        
                
                #as
                A_y[k,k-nx] = -(Sfy/dy)*(2*mu + Lambda)

                #ae                
                A_y[k,k+1] = -(Sfx/dx)*(mu)
                
                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy)
                            + (Sfy/dy)*(4*mu +2*Lambda)*v_top
                            + (Sfx/dx)*(2*mu)*v_left
                            + (Sfy/(3*dx))*Lambda*(
                                                    - u_previous[k+1, 0]
                                                    -  u_previous[k+1-nx, 0]
                                                    + 2*u_left
                                                    )
                            + (Sfx/(3*dy))*mu*(
                                                - u_previous[k-nx , 0]
                                                - u_previous[k+1-nx , 0]
                                                + 2*u_top
                                                )
                        )    
                
            #Top right corner coefficients
                
            elif k == (nx*ny)-1:
                #ap
                
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(6*mu + 3*Lambda) + (Sfx/dx)*(3*mu) 
                
                #as
                A_y[k,k-nx] = -(Sfy/dy)*(2*mu + Lambda)

                #aw                
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                #b_y = A + B(v_t) + C(v_r) + D(+ u_w + u_sw - 2u_r) + E(+ u_s + u_sw - 2u_t) 
                
                b_y[k] = (
                                (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy)   # A
                                + (Sfy/dy)*(4*mu +2*Lambda)*v_top  # + B(v_t)
                                + (Sfx/dx)*(2*Lambda)*v_right      # + C(v_r)
                                + (Sfy/(3*dx))*Lambda*( 
                                                        + u_previous[k-1,0]        # + u_w
                                                        + u_previous[k-1-nx, 0]    # + u_sw
                                                        - 2*u_right                # - 2u_r
                                                        ) 
                                + (Sfx/(3*dy))*mu*(
                                                    + u_previous[k-nx,0]        # + u_s
                                                    + u_previous[k-1-nx, 0]     # + u_sw
                                                    - 2*u_top                   # - 2u_t
                                                    )
                            )
            
            # Center Bottom Boundaries

            elif (k > 0) & (k < nx):

                # print("y - Bottom Center Coefficents for Cell: " + str(k))

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(6*mu + 3*Lambda) + (Sfx/dx)*(2*mu)

                #an
                A_y[k,k+nx] = -(Sfy/dy)*(2*mu + Lambda)
                
                #ae
                A_y[k,k+1] = -(Sfx/dx)*(mu)

                #aw
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                #b_y = A + B(v_b) + C(+ u_ne - u_nw)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            + (Sfy/dy)*(4*mu + 2*Lambda)*v_bottom # v_b
                            + (
                                (Sfy/(4*dx))*Lambda 
                                +
                                (Sfx/(3*dy))*mu
                            )*(
                                + u_previous[k+1+nx, 0] # + u_ne
                                - u_previous[k-1+nx, 0] # - u_nw
                            )
                        )

            # Center Top Boundaries

            elif ( k > (nx*ny)-(nx )) & ( k < (nx*ny)-1 ) :

                # print("y - Top Center Coefficents for Cell: " + str(k))

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(6*mu + 3*Lambda) + (Sfx/dx)*(2*mu)

                #as
                A_y[k,k-nx] = -(Sfy/dy)*(2*mu + Lambda)
                
                #ae
                A_y[k,k+1] = -(Sfx/dx)*(mu)

                #aw
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                #b_y = A + B(v_t) + C(- u_se + u_sw)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            + (Sfy/dy)*(4*mu + 2*Lambda)*v_top # v_t
                            + (
                                (Sfy/(4*dx))*Lambda 
                                +
                                (Sfx/(3*dy))*mu
                            )*(
                                - u_previous[k+1-nx, 0] # - u_se
                                + u_previous[k-1-nx, 0] # + u_sw
                            )
                        )

            # Center Left Boundaries

            elif ( k > 0) & ( k%nx == 0 ) & ( k < (nx*ny)-(nx) ):

                # print("x - Left Center Coefficents for Cell: " + str(k))

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(4*mu + 2*Lambda) + (Sfx/dx)*(3*mu)

                #an
                A_y[k,k+nx] = -(Sfy/dy)*(2*mu + Lambda)
                
                #as
                A_y[k,k-nx] = -(Sfy/dy)*(2*mu + Lambda)

                #ae
                A_y[k,k+1] = -(Sfx/dx)*(mu)

                #b_y = A + B(v_l) + C( + u_ne - u_se)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            + (Sfx/dx)*(2*mu)*v_left # v_l
                            + (
                                (Sfy/(3*dx))*Lambda 
                                +
                                (Sfx/(4*dy))*mu
                            )*(
                                + u_previous[k+1+nx, 0] # + u_ne
                                - u_previous[k+1-nx, 0] # - u_se
                            )
                        )

            # Center Right Boundaries

            elif ( k > nx - 1) & ( k%(nx) == nx - 1 ) & ( k < nx*ny -1 ):

                # print("x - Right Center Coefficents for Cell: " + str(k))

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(4*mu + 2*Lambda) + (Sfx/dx)*(3*mu)

                #an
                A_y[k,k+nx] = -(Sfy/dy)*(2*mu + Lambda)
                
                #as
                A_y[k,k-nx] = -(Sfy/dy)*(2*mu + Lambda)

                #aw
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                #b_y = A + B(v_r) + C( - u_nw + u_sw)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            + (Sfx/dx)*(2*mu)*v_right # v_r
                            + (
                                (Sfy/(3*dx))*Lambda 
                                +
                                (Sfx/(4*dy))*mu
                            )*(
                                - u_previous[k-1+nx, 0] # - u_nw
                                + u_previous[k-1-nx, 0] # + u_sw
                            )
                        )

            else:

                b_y[k] = (
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy)
                            + (Sfy/(4*dx))*Lambda*(
                                                + u_previous[k+1+nx,0]    # + u_ne
                                                - u_previous[k-1+nx,0]    # - u_nw
                                                - u_previous[k+1-nx,0]    # - u_se
                                                + u_previous[k-1-nx,0]    # + u_sw
                                            )
                            +(Sfx/(4*dy))*mu*(
                                                    + u_previous[k+1+nx,0]    # + u_ne
                                                    - u_previous[k-1+nx,0]    # - u_nw
                                                    - u_previous[k+1-nx,0]    # - u_se
                                                    + u_previous[k-1-nx,0]    # + u_sw 
                                                )
                            )
        saveArray(str(time) + "/A_y_" + str(icorr) , A_y.todense())   
        saveArray(str(time) + "/b_y_" + str(icorr), b_y)     
               
                
        ## Solve the y-equation
        Uy = spsolve(A_y, b_y)
        Uy = np.array(Uy)

        saveArray(str(time) + "/Uy_" + str(icorr), Uy)
        
        # ## Update u_new with new x and y displacements

        u_new1 = np.vstack((Ux, Uy)).T
        saveArray (str(time) + "/u_new1_" + str(icorr), u_new1)    
        saveArray (str(time) + "/u_previous_" + str(icorr), u_previous)

        # # u_new[:,0] = Ux
        # # u_new[:,1] = Uy
        
        ## Calculate the residual of each iteration
        
        normFactor = np.max(u_new1)
        residual = math.sqrt(np.mean((u_new1 - u_previous)**2))/normFactor
        tolerance = 1e-6

        u_new = u_new1
    
        # #Print out residual of each iteration
        print("icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}".format(icorr, residual, normFactor))
             
        ## Convergence check
        
        if residual < tolerance:
            
            break
            
        elif icorr > maxcorr:
            
            break
            
        
        icorr = icorr + 1
    
    ## Update variables
    
    u_old_old = u_old
    u_old = u_new
    
    ## Add the boundary conditions to the solution matrices
    u_reshape_x = u_new[:,0].reshape(nx, ny)
    u_reshape_y = u_new[:,1].reshape(nx, ny)
    
    #Boundary conditions for u matrix
    U = np.zeros((nx+2,ny+2))
    U[0,1:nx+1] = u_bottom
    U[1:ny+1,0] = u_left
    U[1:ny+1,nx+1] = u_right
    U[ny+1,1:nx+1] = u_top    
    U[1:nx+1,1:nx+1] = u_reshape_x
    
    # #Boundary conditions for v matrix
    V = np.zeros((nx+2,ny+2))
    V[0,1:nx+1] = v_bottom
    V[1:ny+1,0] = v_left
    V[1:ny+1,nx+1] = v_right
    V[ny+1,1:nx+1] = v_top    
    V[1:nx+1,1:nx+1] = u_reshape_y  
    
    #Modulus of displacement
    disp_mod = np.sqrt((U**2) + (V**2))
    disp_mod_converted = disp_mod*1e6 #Convert displacement to micrometers
    
    # ## Plot the results

    import plotly.graph_objects as go

    disp_mod = np.sqrt((U**2) + (V**2))*1e-6

    dispx, dispy = np.meshgrid(x, y)
    fig = go.Figure(data=[go.Surface(z=disp_mod_converted, x=dispx, y=dispy)])
    
fig.show()   




Time = 1
-2640000000000.0
icorr = 1, Residual = 0.51381969393152016057, normFactor = 0.00000665908276492223


In [31]:
A_x[1,2]

-780000000000.0